In [1]:
import pandas as pd
from datetime import datetime

In [2]:
cmsa_df_filled = pd.read_csv('../data/cmsa_small_filled.csv')

In [3]:
cmsa_df_zero = pd.read_csv('../data/cmsa_small_fill_zero.csv')

### Merge w. vacation, holidays & COVID stringency

In [4]:
vacation_df = pd.read_csv('../data/vacation.csv')

In [5]:
holiday_df = pd.read_csv('../data/holiday.csv')

In [6]:
covid_string = pd.read_csv('../data/covid-stringency.csv')

In [7]:
new_row = {'Day':'2022-01-01', 'stringency_index':63.89}
new_row2 = {'Day':'2022-01-02', 'stringency_index':63.89}
covid_string = covid_string.append(new_row, ignore_index=True)
covid_string = covid_string.append(new_row2, ignore_index=True)

In [8]:
covid_string

,Day,stringency_index
0,2020-09-01,50.93
1,2020-09-02,50.93
2,2020-09-03,50.93
3,2020-09-04,50.93
4,2020-09-05,50.93
...,...,...
484,2021-12-29,63.89
485,2021-12-30,63.89
486,2021-12-31,63.89
487,2022-01-01,63.89


In [9]:
def date_time_col(df):
    df["datetime_utc"] = pd.to_datetime(df["datetime"], utc = True)
    df["datetime"] = df["datetime_utc"].dt.tz_convert("Europe/Amsterdam")
    del df["datetime_utc"]
    df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.sort_values(by = "datetime", ascending = True)
    return df

In [10]:
vacation_df = date_time_col(vacation_df)
holiday_df = date_time_col(holiday_df)

In [11]:
covid_string["Day"] = pd.to_datetime(covid_string["Day"], utc = True)
covid_string['Day'] = covid_string['Day'].dt.strftime('%Y-%m-%d %H:%M:%S')
covid_string = covid_string.rename(columns={"Day": "datetime"})

In [12]:
covid_string

,datetime,stringency_index
0,2020-09-01 00:00:00,50.93
1,2020-09-02 00:00:00,50.93
2,2020-09-03 00:00:00,50.93
3,2020-09-04 00:00:00,50.93
4,2020-09-05 00:00:00,50.93
...,...,...
484,2021-12-29 00:00:00,63.89
485,2021-12-30 00:00:00,63.89
486,2021-12-31 00:00:00,63.89
487,2022-01-01 00:00:00,63.89


In [13]:
cmsa_df_filled = cmsa_df_filled.set_index('datetime')
cmsa_df_zero = cmsa_df_zero.set_index('datetime')
vacation_df = vacation_df.set_index('datetime')
holiday_df = holiday_df.set_index('datetime')
covid_string = covid_string.set_index('datetime')

In [14]:
cmsa_df_filled.head()

,GAWW-11,GAWW-12,GAWW-14
datetime,,,
2020-09-01 00:00:00,104.0,113.0,32.0
2020-09-01 00:15:00,73.0,109.0,30.0
2020-09-01 00:30:00,84.0,88.0,27.0
2020-09-01 00:45:00,95.0,99.0,26.0
2020-09-01 01:00:00,63.0,84.0,43.0


In [15]:
cmsa_df_filled.index = pd.DatetimeIndex(cmsa_df_filled.index)
cmsa_df_zero.index = pd.DatetimeIndex(cmsa_df_zero.index)

In [16]:
vacation_df.index = pd.DatetimeIndex(vacation_df.index)
holiday_df.index = pd.DatetimeIndex(holiday_df.index)
covid_string.index = pd.DatetimeIndex(covid_string.index)

In [17]:
start_date = '2020-09-01 00:00:00'
end_date = '2022-01-01 23:45:00'

In [18]:
cmsa_df_filled.shape

(46848, 3)

In [19]:
holiday_df = holiday_df.resample('15min').ffill(limit=96)
vacation_df = vacation_df.resample('15min').ffill(limit=96)
covid_string = covid_string.resample('15min').ffill(limit=96)

In [20]:
vacation_df = vacation_df[(vacation_df.index >= start_date) & (vacation_df.index <= end_date)]
holiday_df = holiday_df[(holiday_df.index >= start_date) & (holiday_df.index <= end_date)]
covid_string = covid_string[(covid_string.index >= start_date) & (covid_string.index <= end_date)]

In [21]:
cmsa_df_filled_merged = pd.concat([cmsa_df_filled, vacation_df, holiday_df, covid_string], axis=1)

In [22]:
cmsa_df_fill_zero_merged = pd.concat([cmsa_df_zero, vacation_df, holiday_df, covid_string], axis=1)

In [23]:
cmsa_df_filled_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index
datetime,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93
...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89


In [24]:
cmsa_df_fill_zero_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index
datetime,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93
...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89


In [25]:
cmsa_df_filled_merged.to_csv('../data/cmsa_df_filled_merged.csv')

In [26]:
cmsa_df_fill_zero_merged.to_csv('../data/cmsa_df_fill_zero_merged.csv')

### Merge w. GVB

In [102]:
gvb_dam = pd.read_csv("../data/gvb/gvb_dam.csv", index_col="Unnamed: 0")
gvb_nieuwmarkt = pd.read_csv("../data/gvb/gvb_nieuwmarkt.csv", index_col="Unnamed: 0")
gvb_dam.rename(columns={'checkin':'checkin_dam', 'checkout':'checkout_dam'}, inplace=True)
gvb_dam.drop(['stop_name', 'date', 'hour'], axis=1, inplace=True)
gvb_nieuwmarkt.rename(columns={'checkin':'checkin_nieuwmarkt', 'checkout':'checkout_nieuwmarkt'}, inplace=True)
gvb_nieuwmarkt.drop(['stop_name', 'date', 'hour'], axis=1, inplace=True)

In [103]:
cmsa_merged = pd.read_csv("../data/cmsa_df_filled_merged.csv")
cmsa_fill_zero_merged = pd.read_csv("../data/cmsa_df_fill_zero_merged.csv")

In [104]:
cmsa_fill_zero_merged.shape

(46848, 7)

In [106]:
cmsa_merged = cmsa_merged.set_index('datetime')
cmsa_fill_zero_merged = cmsa_fill_zero_merged.set_index('datetime')

In [107]:
gvb_dam = gvb_dam.set_index('datetime')
gvb_nieuwmarkt = gvb_nieuwmarkt.set_index('datetime')

In [108]:
cmsa_merged.index = pd.DatetimeIndex(cmsa_merged.index)
cmsa_fill_zero_merged.index = pd.DatetimeIndex(cmsa_fill_zero_merged.index)
gvb_dam.index = pd.DatetimeIndex(gvb_dam.index)
gvb_nieuwmarkt.index = pd.DatetimeIndex(gvb_nieuwmarkt.index)

In [109]:
cmsa_merged.head()

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index
datetime,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93


In [110]:
gvb_dam.head()

,checkin_dam,checkout_dam
datetime,,
2020-09-01 00:00:00,48.0,NaN
2020-09-01 01:00:00,12.0,NaN
2020-09-01 02:00:00,23.0,NaN
2020-09-01 06:00:00,NaN,32.0
2020-09-01 07:00:00,NaN,61.0


#### Merge and fill every null by 0

In [111]:
gvb_dam_every_15min = gvb_dam.resample('15min').ffill(limit=4)

In [112]:
gvb_nieuwmarkt_every_15min = gvb_nieuwmarkt.resample('15min').ffill(limit=4)

In [113]:
cmsa_fill_zero_merged_v2 = pd.concat([cmsa_fill_zero_merged, gvb_dam_every_15min, gvb_nieuwmarkt_every_15min], axis=1)

In [ ]:
cmsa_fill_zero_merged_v2.head()

In [115]:
cmsa_fill_zero_merged_v2.fillna(0, inplace=True)

In [116]:
cmsa_fill_zero_merged_v2.head()

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index,checkin_dam,checkout_dam,checkin_nieuwmarkt,checkout_nieuwmarkt
datetime,,,,,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93,48.0,0.0,58.0,17.0
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93,48.0,0.0,58.0,17.0
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93,48.0,0.0,58.0,17.0
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93,48.0,0.0,58.0,17.0
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93,12.0,0.0,58.0,17.0


In [117]:
cmsa_fill_zero_merged_v2.shape

(46848, 10)

In [118]:
cmsa_fill_zero_merged_v2.to_csv("../data/cmsa_fill_zero_merged_v2.csv")

### Proper data cleaning

In [119]:
cmsa_merged_v2 = pd.concat([cmsa_merged, gvb_dam_every_15min, gvb_nieuwmarkt_every_15min], axis=1)

In [120]:
gvb_dam.tail()

,checkin_dam,checkout_dam
datetime,,
2021-12-12 19:00:00,169.0,NaN
2021-12-12 20:00:00,54.0,NaN
2021-12-12 21:00:00,54.0,NaN
2021-12-12 22:00:00,53.0,NaN
2021-12-12 23:00:00,13.0,NaN


In [121]:
gvb_nieuwmarkt.tail()

,checkin_nieuwmarkt,checkout_nieuwmarkt
datetime,,
2021-12-12 19:00:00,105.0,NaN
2021-12-12 20:00:00,77.0,NaN
2021-12-12 21:00:00,47.0,NaN
2021-12-12 22:00:00,40.0,NaN
2021-12-12 23:00:00,15.0,NaN


In [122]:
cmsa_merged_v2['time'] = cmsa_merged_v2.index.time

In [123]:
cmsa_merged_v2.head()

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index,checkin_dam,checkout_dam,checkin_nieuwmarkt,checkout_nieuwmarkt,time
datetime,,,,,,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93,48.0,NaN,58.0,17.0,00:00:00
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93,48.0,NaN,58.0,17.0,00:15:00
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93,48.0,NaN,58.0,17.0,00:30:00
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93,48.0,NaN,58.0,17.0,00:45:00
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93,12.0,NaN,58.0,17.0,01:00:00


In [124]:
# total count and percentage of missing values per column
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(cmsa_merged_v2)

,Total,Percent
checkout_dam,18448,39.38
checkin_dam,18239,38.93
checkin_nieuwmarkt,15916,33.97
checkout_nieuwmarkt,14928,31.86


In [125]:
import numpy as np

In [126]:
# df['Normalized'] = df['Budget'].where(df['Currency']=='$', df['Budget'] * 0.78125)
# features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mean()[0]))
# df.groupby('Plate')['LogRatio'].transform(lambda s: s.loc[[True if v < s.quantile(q=0.8) and v > s.quantile(q=0.2) else False for v in s]].mean())

cmsa_merged_v2['checkout_dam'] = cmsa_merged_v2.groupby('time')['checkout_dam'].transform(lambda x: x.fillna(x.mean()))
cmsa_merged_v2['checkin_dam'] = cmsa_merged_v2.groupby('time')['checkin_dam'].transform(lambda x: x.fillna(x.mean()))
cmsa_merged_v2['checkin_nieuwmarkt'] = cmsa_merged_v2.groupby('time')['checkin_nieuwmarkt'].transform(lambda x: x.fillna(x.mean()))
cmsa_merged_v2['checkout_nieuwmarkt'] = cmsa_merged_v2.groupby('time')['checkout_nieuwmarkt'].transform(lambda x: x.fillna(x.mean()))

In [127]:
# total count and percentage of missing values per column
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(cmsa_merged_v2)

,Total,Percent
checkin_nieuwmarkt,9272,19.79
checkout_nieuwmarkt,9272,19.79
checkout_dam,5856,12.50


In [128]:
cmsa_merged_v2.fillna(0, inplace=True)

In [129]:
# total count and percentage of missing values per column
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(cmsa_merged_v2)

,Total,Percent


In [131]:
cmsa_merged_v2 = cmsa_merged_v2.round({'checkin_dam': 0, 'checkout_dam': 0, 'checkin_nieuwmarkt': 0, 'checkout_nieuwmarkt': 0})

In [ ]:
cmsa_merged_v2.drop(['time'], axis=1, inplace=True)


In [134]:
cmsa_merged_v2.to_csv("../data/cmsa_merged_v2.csv")

In [3]:
cmsa_gvb = pd.read_csv("../data/cmsa_gvb_merged.csv", index_col='datetime')
cmsa_covid_hotel = pd.read_csv("../data/cmsa_covid_hotel_merged.csv", index_col='datetime')

In [5]:
cmsa_covid_hotel.columns

Index(['GAWW-11', 'GAWW-12', 'GAWW-14', 'vacation_dummy', 'holiday_dummy',
       'stringency_index', 'hotel_gasten', 'hotel_overnachtingen',
       'airport_tot_passengers', 'airport_arrived_passengers',
       'airport_departed_passengers', 'covid_cases', 'covid_hospital',
       'covid_deaths'],
      dtype='object')

In [6]:
cmsa_gvb.columns

Index(['GAWW-11', 'GAWW-12', 'GAWW-14', 'vacation_dummy', 'holiday_dummy',
       'stringency_index', 'checkin_dam', 'checkout_dam', 'checkin_nieuwmarkt',
       'checkout_nieuwmarkt'],
      dtype='object')

In [14]:
cmsa_covid_hotel = cmsa_covid_hotel[['hotel_gasten', 'hotel_overnachtingen',
       'airport_tot_passengers', 'airport_arrived_passengers',
       'airport_departed_passengers', 'covid_cases', 'covid_hospital',
       'covid_deaths']]

In [15]:
cmsa_merged_v2 = pd.concat([cmsa_gvb, cmsa_covid_hotel], axis=1)

In [16]:
cmsa_merged_v2.columns

Index(['GAWW-11', 'GAWW-12', 'GAWW-14', 'vacation_dummy', 'holiday_dummy',
       'stringency_index', 'checkin_dam', 'checkout_dam', 'checkin_nieuwmarkt',
       'checkout_nieuwmarkt', 'hotel_gasten', 'hotel_overnachtingen',
       'airport_tot_passengers', 'airport_arrived_passengers',
       'airport_departed_passengers', 'covid_cases', 'covid_hospital',
       'covid_deaths'],
      dtype='object')

In [19]:
cmsa_merged_v2.to_csv("../data/cmsa_combined_without_knmi.csv")